In [95]:
import mlflow


mlflow.__version__

'1.26.1'

In [4]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = 'sqlite:///mlflow.db'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [5]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [6]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string='',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=['metrics.rmse ASC']

)

In [51]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: c458c2f0cd6f498899439f5aff540282, rmse: 4.2979
run id: 93590964ca5d4303b53c706291af04dd, rmse: 5.8789


In [42]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Production
version: 2, stage: Staging


In [43]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1653865269364, current_stage='Production', description='', last_updated_timestamp=1653870776082, name='nyc-taxi-regressor', run_id='c458c2f0cd6f498899439f5aff540282', run_link='', source='./mlruns/1/c458c2f0cd6f498899439f5aff540282/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>

In [88]:
import pandas as pd
import mlflow
from sklearn.metrics import mean_squared_error



def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

def test_model(run_id, X_test, y_test):
    model = mlflow.xgboost.load_model(f"runs:/{run_id}/models_mlflow")
    y_pred = model.predict(X_test)
    return {"rmse:": mean_squared_error(y_test, y_pred, squared=False)}







In [89]:
df = read_dataframe('https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-03.parquet')
df.shape

(80372, 21)

In [90]:
run_id = 'c458c2f0cd6f498899439f5aff540282'
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'/home/ubuntu/notebooks/week 2/preprocessor'

In [91]:
import pickle

with open('preprocessor/preprocessor.b', 'rb') as f_in:
    dv = pickle.load(f_in)




In [92]:
X_test = preprocess(df, dv)

In [93]:
target = 'duration'
y_test = df[target].values

In [94]:
%time test_model(run_id=run_id, X_test=X_test, y_test=y_test)

MissingConfigException: Yaml file '/home/ubuntu/notebooks/week 2/mlruns/1/c458c2f0cd6f498899439f5aff540282/meta.yaml' does not exist.